### 0. Start by retrieving input & formatting it into a nparray

In [2]:
with open('advent-6-0-input.txt') as f:
    map = f.read().splitlines()

with open('advent-6-0-inputex.txt') as f:
    ex = f.read().splitlines()

ex

['....#.....',
 '.........#',
 '..........',
 '..#.......',
 '.......#..',
 '..........',
 '.#..^.....',
 '........#.',
 '#.........',
 '......#...']

In [3]:
# convert sequence of symbols into a list
def separate_symbol(liste):
    list_ = []
    for i in range(len(liste)):
        line = list(liste[i])
        list_.append(line)
    return list_

separate_symbol(ex)

[['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']]

In [4]:
split_ex = separate_symbol(ex)
split_map = separate_symbol(map)

In [5]:
# Convert data to a matrix (np.array of dim 2)
import numpy as np

np_ex = np.array(split_ex)
np_ex, np_ex.ndim

(array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
        ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U1'),
 2)

In [6]:
np_map = np.array(split_map)
np_map.shape

(130, 130)

### 1. Get positions

In [7]:
# Find where the guard and obstacles are
def get_coord(np_, str_):
    x, y = np.where(np_ == str_)
    return list(zip(x,y))

get_coord(np_ex,'^'), get_coord(np_ex,'#')

([(6, 4)], [(0, 4), (1, 9), (3, 2), (4, 7), (6, 1), (7, 8), (8, 0), (9, 6)])

In [8]:
obstacles_ex = get_coord(np_ex,'#')
obstacles = get_coord(np_map,'#')

In [9]:
np_ex[6,:] # sanity check -> 1st argument is the n-th LINE (and not column)

array(['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'], dtype='<U1')

In [10]:
np.where(np_ex[8,:] == '#')[0][0], type(np.where(np_ex[6,:] == '#')[0][0]) # where: returns position of found elements

(0, numpy.int64)

In [11]:
len(np.where(np_ex[6,:] == '.')[0]) # returns nb of occurrences found

8

In [12]:
def get_all_pos(np_): # to be used in below function ?? We will use for now just a count instead of retrieving coordinates
    x = np.where(np_ != " ")
    return x

get_all_pos(np_ex[6,:]), get_all_pos(np_ex[:1,0]) # same output whether np in input is by row or column

((array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),), (array([0]),))

In [22]:
def get_positions(np_):
    direction = 'up'
    x, y = np.where(np_ == '^') # initial guard's position
    x = x[0]
    y = y[0]
    blocked = False
    size = np_.shape[0]
    # list_ = []
    # positions_to_add = []
    count_positions = 1 # we count the initial position
    i = 0
    while not blocked:
        i += 1
        if direction == 'up':
            # print(np.where(np_[:x,y] == '#')[0][0])
            index_ = np.where(np_[:x,y] == '#')[0][0] 
            print(f"UP - guard at ({x}, {y}) VS obstacle at ({index_}, {y}) (index = {index_})")
            if index_ == -1: # no obstacle found: go to y = 0
                # positions_to_add = [x in np_[0:x+1, y]] # y+1 because upper bound is EXCLUSIVE
                count_ = len(np.where(np_[:x,y] == '.')[0])
                x = 0
                print("UP if")
            elif index_ == x-1:
                blocked = True
                print("UP elif")
                break
            else:
                # positions_to_add = np_[index_+1:x+1, y]
                print("UP else")
                count_ = len(np.where(np_[index_+1:x,y] == '.')[0])
                x = index_+1 # guard moves to the location just before the obstacle
            print(f"UP out of if from direction = {direction}")
            direction = 'right'
            print(f"to {direction}")

        elif direction == 'right':
            index_ = np.where(np_[x,y:] == '#')[0][0] 
            print(f"RIGHT - guard at ({x}, {y}) VS obstacle at ({x}, {index_+y}) (index = {index_})")
            if index_ == -1: # no obstacle found: go to x = len-1
                # positions_to_add = np_[x, y:size] 
                count_ = len(np.where(np_[x,y:] == '.')[0])
                y = size - 1
            elif index_+y == y+1: # ATTENTION: ajouter +y car index_ compté à partir de y
                blocked = True
                break
            else:
                # positions_to_add = np_[x,y:index_] 
                count_ = len(np.where(np_[x,y+1:index_] == '.')[0])
                y = index_-1 +y
            direction = 'down'

        elif direction == 'down':
            index_ = np.where(np_[x:,y] == '#')[0][0] 
            print(f"DOWN - guard at ({x}, {y}) VS obstacle at ({index_+x}, {y}) (index = {index_})")
            if index_ == -1: # no obstacle found: go to y = len-1
                # positions_to_add = np_[x:size, y] 
                count_ = len(np.where(np_[x:,y] == '.')[0])
                x = size - 1
            elif index_ +x == x+1: # ATTENTION: ajouter +x car index_ compté à partir de x
                blocked = True
                break
            else:
                # positions_to_add = np_[x:index_, y]
                count_ = len(np.where(np_[x+1:index_,y] == '.')[0])
                x = index_-1+x
            direction = 'left'

        elif direction == 'left':
            index_ = np.where(np_[x,:y] == '#')[0][0] 
            print(f"LEFT - guard at ({x}, {y}) VS obstacle at ({x}, {index_}) (index = {index_})")
            if index_ == -1: # no obstacle found: go to x = 0
                # positions_to_add = np_[x,0:y]  # y+1 because upper bound is EXCLUSIVE
                count_ = len(np.where(np_[x,:y] == '.')[0])
                y = 0
            elif index_ == y-1:
                blocked = True
                break
            else:
                # positions_to_add = np_[x, index_+1:y+1]
                count_ = len(np.where(np_[x,index_+1:y] == '.')[0])
                y = index_+1
            direction = 'up'
        
        count_positions += count_
        # list_.extend(positions_to_add)
        
    return count_positions, i
    # print(list_)
    # return len(set(list_)) # we use set() to remove duplicates if any

get_positions(np_ex) # expected : 41

UP - guard at (6, 4) VS obstacle at (0, 4) (index = 0)
UP else
UP out of if from direction = up
to right
RIGHT - guard at (1, 4) VS obstacle at (1, 9) (index = 5)
DOWN - guard at (1, 8) VS obstacle at (7, 8) (index = 6)
LEFT - guard at (6, 8) VS obstacle at (6, 1) (index = 1)
UP - guard at (6, 2) VS obstacle at (3, 2) (index = 3)
UP else
UP out of if from direction = up
to right
RIGHT - guard at (4, 2) VS obstacle at (4, 7) (index = 5)
DOWN - guard at (4, 6) VS obstacle at (9, 6) (index = 5)
LEFT - guard at (8, 6) VS obstacle at (8, 0) (index = 0)
UP - guard at (8, 1) VS obstacle at (6, 1) (index = 6)
UP else
UP out of if from direction = up
to right
RIGHT - guard at (7, 1) VS obstacle at (7, 8) (index = 7)


IndexError: index 0 is out of bounds for axis 0 with size 0